**Copyright 2023 Google LLC.**

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2023 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img align="left" width="150" src="https://services.google.com/fh/files/misc/ml_toast_logo.png" alt="ml_toast_logo" /><br><br>

# 🍞 ML-ToAST: **M**ulti**l**ingual **To**pic Clustering of **A**ds-triggering **S**earch **T**erms

**Disclaimer: This is not an official Google product.**

**🍞 ML-ToAST** is an open-source tool that helps users cluster multilingual search terms captured from different time windows into semantically relevant topics. It helps advertisers / marketers surface insights related to changing consumer interest in a configurable, user-friendly, and privacy-safe manner.

More information available at [github.com/google/ml_toast](https://github.com/google/ml_toast).

## Prerequisites

### Which search terms to use?

We recommend extracting the Google Ads [Search Terms report](https://support.google.com/google-ads/answer/2472708) for the following periods:
 * **Last 30 days** (e.g. Nov 1 - Nov 30): it generally makes sense to look at the most recent search terms that triggered your ads.
 * **Previous 30/31 days** (e.g. Oct 1 - Oct 31): this helps provide information on those search terms that constitute your core business over those that are recently trending.
 * **Last 30 days last year** (e.g. Nov 1 - Nov 30 of the previous year): to account for seasonality effects (e.g. holiday season).

We also recommend restricting the extracted search terms to a subset of *related* campaigns (e.g. all campaigns for a specific *product line* or *operating domain*) rather than all campaigns in your account. This allows the model to better capture how the search terms relate to one another, and therefore, extract more meaningful topics.

The report can be downloaded from the Google Ads UI in CSV format and imported into a Google Sheets spreadsheet, to be used below for input/output.

*Note: if you have multiple accounts operating under the same product line or domain, you can extract search terms from those accounts as well and group them all into the same Google Sheets spreadsheet.*

## Get Started

In [ ]:
#@title Authenticate your user for this colab session
import logging
from google.colab import auth

auth.authenticate_user()
logging.getLogger().setLevel(logging.INFO)

In [ ]:
#@title Install dependencies
!pip install tensorflow-text hdbscan umap-learn

## Input and Preprocessing


In [ ]:
#@title Configurable params { run: 'auto' }

#@markdown Enter your spreadsheet ID:
spreadsheet_id = "id-goes-here" #@param {type:"string"}

#@markdown Enter the main worksheet name (which should usually contain the search terms from the last month):
input_sheet_name = "colab-input-main" #@param {type:"string"}

#@markdown Enter the prefix for any additional worksheets you would also like to analyze (e.g. search terms of the previous month, previous year, etc.):
additional_sheets_prefix = "colab-input-lookback-" #@param {type:"string"}

#@markdown Enter the name of the column that contains search terms. This value should be the same across all worksheets:
search_terms_column = "Search term" #@param {type:"string"}

#@markdown <hr>Filtering settings

#@markdown ***Check*** the checkbox to filter on new terms (i.e. compare search terms from the aforementioned lookback worksheets) and ***uncheck*** to analyze search terms from the main worksheet only.
filter_new_terms = True #@param {type:"boolean"}

#@markdown Enter the name of a column that contains a metric you would like to use for filtering and/or sorting (e.g. impressions):
filter_metric_column = "Impr." #@param {type:"string"}

#@markdown Set this to filter terms with a *metric* (e.g. impressions) value lower than the input. Set to *-1* to skip filtering.
filter_metric_max_threshold = -1 #@param {type:"integer"}

#@markdown Select the desired logical grouping (AND/OR) of the filters:
filters_grouping = "AND" #@param ["AND", "OR"]

#@markdown <hr>Advanced settings

#@markdown Enter a comma-separated list of *stop words* which should be excluded from all generated topics:
stop_words = "stop1, stop2" #@param {type:"string"}

if stop_words:
  stop_words = stop_words.replace(', ', ',').split(',')
else:
  stop_words = None

#@markdown ***Check*** the checkbox to perform hyperparameter tuning for UMAP + HDBSCAN (increases processing time by a factor of ~3).<br>
#@markdown Despite the time factor, we **highly** recommend using this to provide the optimal results for the given input.<br>
#@markdown ***Uncheck*** to use the default clustering parameters.
hyperparameter_tuning = False #@param {type:"boolean"}

# Google Ads specific params
match_type_column = "Match type"
match_type_broad = "Broad match"
known_report_metrics = ['Clicks', 'Impr.', 'Cost']
all_report_metrics = (
    known_report_metrics if filter_metric_column in known_report_metrics
    else [filter_metric_column])

# Validation rules
if not spreadsheet_id or not input_sheet_name or not search_terms_column:
  raise ValueError(
      'Invalid input! Please make sure at least '
      '"spreadsheet_id", "input_sheet_name" and "search_terms_column" '
      'are provided.')

# Debugging params
# Calculates performance metrics for the output topics, which will be done in
# the provided LookerStudio dashboard (set to True if not using the dashboard)
output_topic_metrics = False

In [ ]:
#@title Fetch data from the input spreadsheet
#@markdown The first row in each worksheet will be considered the **column headers** row.
import pandas as pd
import gspread
from google.auth import default

creds, _ = default()
sheets_client = gspread.authorize(creds)
spreadsheet = sheets_client.open_by_key(spreadsheet_id)

input_values = spreadsheet.worksheet(input_sheet_name).get_all_values()
additional_sheets_values = []

if filter_new_terms and additional_sheets_prefix:
  for sheet in spreadsheet.worksheets():
    if sheet.title.startswith(additional_sheets_prefix):
      additional_sheets_values.append(sheet.col_values(1))

input_data = pd.DataFrame(input_values[1:], columns=input_values[0])

for report_metric in all_report_metrics:
  if report_metric in input_data.columns:
    input_data[report_metric] = pd.to_numeric(
        input_data[report_metric].str.replace(',', ''))

print(
    f'Worksheet: {input_sheet_name}\nNumber of rows: {len(input_data)}\n'
    'First 5 rows:')
input_data.head()

In [ ]:
#@title Extract search terms and apply the defined filters

def add_filter(existing_filter, new_filter):
  if filters_grouping == 'AND':
    return existing_filter & new_filter
  return existing_filter | new_filter

additional_data = [
    pd.DataFrame(sheet_values[1:], columns=[sheet_values[0]])
    for sheet_values in additional_sheets_values]

data_unfiltered = input_data.copy()

series_filter = (filters_grouping == 'AND')
applied_filters = []

if additional_data:
  data_unfiltered = (
      data_unfiltered.merge(pd.concat(additional_data).drop_duplicates(),
                    on=search_terms_column,
                    how='left',
                    indicator=True))
  series_filter = add_filter(
      existing_filter=series_filter, new_filter=(
          data_unfiltered['_merge'] == 'left_only'))
  applied_filters.append('filter_new_terms')

if filter_metric_column and filter_metric_max_threshold > 0:
  series_filter = add_filter(
      existing_filter=series_filter, new_filter=(
      data_unfiltered[filter_metric_column] <= filter_metric_max_threshold))
  applied_filters.append(
      f'filter_metric_max_threshold < {filter_metric_max_threshold}')

filtered_data = (
    data_unfiltered[series_filter] if applied_filters else data_unfiltered)

if '_merge' in filtered_data.columns:
  filtered_data = filtered_data.drop(columns='_merge')

if filter_metric_column in filtered_data.columns:
  filtered_data = filtered_data.sort_values(
      by=filter_metric_column, ascending=False)

print('\n'.join([
    f'Filtered data - total number of rows: {len(filtered_data)}',
    f'Filters applied: {applied_filters}',
    (
        f"Filters logical grouping: '{filters_grouping}'"
        if len(applied_filters) > 1 else ''),
    'First 5 rows:']))
filtered_data.head()

In [ ]:
#@title Extract 'Broad Match' and 'non Broad Match' terms from the filtered data

if match_type_column in filtered_data.columns:
  broad_match_terms = filtered_data.copy()
  broad_match_terms = broad_match_terms[
      filtered_data[match_type_column] == match_type_broad]

  non_broad_match_terms = filtered_data.copy()
  non_broad_match_terms = non_broad_match_terms[
      filtered_data[match_type_column] != match_type_broad]

  if filter_metric_column in filtered_data.columns:
    broad_match_terms = broad_match_terms.sort_values(
        by=filter_metric_column, ascending=False)
    non_broad_match_terms = non_broad_match_terms.sort_values(
        by=filter_metric_column, ascending=False)

  broad_match_groups = {
      'filtered_broad_match_terms': broad_match_terms,
      'filtered_non_broad_match_terms': non_broad_match_terms,
  }
  print(
      'Extracted:\n'
      f' - All terms where "{match_type_column}" is "{match_type_broad}" '
      f'from the filtered data. Number of rows: {len(broad_match_terms)}')
  print(
      'Extracted:\n'
      f' - All terms where "{match_type_column}" is NOT "{match_type_broad}" '
      f'from the filtered data. Number of rows: {len(non_broad_match_terms)}')
else:
  broad_match_groups = {}
  print(
      f'No column "{match_type_column}" found in the input data. '
      'Skipping extraction of Broad Match terms.')

## Topic Clustering

In [ ]:
#@title Import the topic clustering library
!echo "Restoring working directory to root..."
%cd /content
!rm -rf ml_toast && git clone https://github.com/google/ml_toast.git
!echo "Changing working directory to ml_toast..."
%cd ml_toast

from ml_toast import topic_clustering as topic_clustering_lib

In [ ]:
#@title Use the library to determine topics for all input groups
terms_input_data_dict = {'filtered_terms': filtered_data}
for key, broad_match_group in broad_match_groups.items():
  terms_input_data_dict[key] = broad_match_group

for key, terms_input_data in terms_input_data_dict.items():
  topic_clustering = topic_clustering_lib.TopicClustering(
      data_id=key,
      input_col=search_terms_column,
      stop_words=stop_words,
      do_hdbscan_hyperopt=hyperparameter_tuning)
  topics_kmeans, topics_hdbscan = topic_clustering.determine_topics(
      terms_input_data)
  terms_input_data['Topic'] = topics_kmeans
  terms_input_data['Additional Topics'] = topics_hdbscan

## Output

In [ ]:
#@title Write results back to the input spreadsheet
#@markdown New worksheets with the prefix **colab-** will be appended to the spreadsheet,
#@markdown or overwritten if they already exist.
from gspread_dataframe import set_with_dataframe

for key, terms_input_data in terms_input_data_dict.items():
  try:
    output_sheet = spreadsheet.worksheet(f'colab-{key}-output')
    output_sheet.clear()
  except gspread.exceptions.WorksheetNotFound:
    output_sheet = spreadsheet.add_worksheet(
        f'colab-{key}-output',
        rows=len(terms_input_data),
        cols=len(terms_input_data.columns))
  set_with_dataframe(
      output_sheet, terms_input_data, include_column_header=True)

  if output_topic_metrics:
    for topics_type in ['Topic', 'Additional Topics']:
      cluster_metrics = pd.DataFrame()
      cluster_metrics['Topic'] = terms_input_data[topics_type]
      for report_metric in all_report_metrics:
        if report_metric in terms_input_data.columns:
          cluster_metrics[report_metric] = terms_input_data[report_metric]

      cluster_metrics = cluster_metrics.groupby(by='Topic', sort=False).agg(
          ['mean', 'median', 'min', 'max', 'std', 'var'])
      cluster_metrics.insert(loc=0, column='Topic', value=cluster_metrics.index)
      cluster_metrics.insert(
          loc=1,
          column='Count',
          value=terms_input_data.groupby(by=topics_type, sort=False).count()[
              search_terms_column])
      cluster_metrics = cluster_metrics.sort_values(by='Count', ascending=False)
      cluster_metrics_key = (
          f"{key}_{topics_type.lower().replace(' ', '_')}")

      try:
        metrics_sheet = spreadsheet.worksheet(
            f'colab-{cluster_metrics_key}-metrics')
        metrics_sheet.clear()
      except gspread.exceptions.WorksheetNotFound:
        metrics_sheet = spreadsheet.add_worksheet(
            f'colab-{cluster_metrics_key}-metrics',
            rows=len(cluster_metrics),
            cols=len(cluster_metrics.columns))
      set_with_dataframe(
          metrics_sheet, cluster_metrics, include_column_header=True)

### Visualize the generated topics in LookerStudio

ML-Toast provides a template LookerStudio dashboard to help you visualize the generated topics and quickly surface insights.

Use [this link](https://lookerstudio.google.com/c/u/0/reporting/8bc2240e-a919-4916-9c7f-daf72f75bf42/preview) to create a copy of the dashboard and get started! All you would need to do is map the data sources used by the dashboard to the spreadsheet used for the input / output above.